In [1]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

## 1. 오디오파일에 자막생성
- response_format="srt"


In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
audio_file = open("data/채용면접_샘플_01.wav", "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="srt",  # 자막 포맷
    temperature=0.0,
)

In [4]:
print(transcript)

1
00:00:00,000 --> 00:00:07,000
지금 생각해보면 가장 기억에 남는 것이 외환위기 때의 경험입니다.

2
00:00:07,000 --> 00:00:13,000
외환위기 때 나라뿐 아니라 회사에서는 달러가 많이 부족했고

3
00:00:13,000 --> 00:00:19,000
우리는 수출을 하기 위해서 원자재라든지 대금지급

4
00:00:19,000 --> 00:00:23,000
혹은 또 우리가 수출한 물건에 대한 대금을 받아야 되는 상황이었습니다.

5
00:00:23,000 --> 00:00:29,000
일단은 우리가 해외로부터 받아야 될 때는 최대한 그것을 달러로 받았고

6
00:00:29,000 --> 00:00:36,000
반대로 우리가 지불해야 될 것은 가능한 한 원자재를 통해서 지급을 했습니다.

7
00:00:36,000 --> 00:00:40,000
그 원자재라는 것이 결국은 반도체 쪽이었는데

8
00:00:40,000 --> 00:00:46,000
우리나라에서 가장 나름 손쉽게 구하면서도 꼭 필요한 제품인 반도체를

9
00:00:46,000 --> 00:00:48,000
원자재값 대응으로 물건을 주면서

10
00:00:48,000 --> 00:00:53,000
반대로 해외에서 우리가 받아야 될 것은 달러로 받으면서

11
00:00:53,000 --> 00:00:57,000
그 환율차를 가장 줄일 수가 있었습니다.

12
00:00:57,000 --> 00:01:06,000
그것으로 인해서 약 6개월 동안에 나름 회사에서는 달러 지출을 막을 수 있었고

13
00:01:06,000 --> 00:01:15,000
그때 그나마 달러를 회사에서 확보를 해서 나름의 위기를 극복할 수 있었던 것으로 생각합니다.





## 2. YouTube url에서 음성파일을 만들기
- pytube 라이브러리 오류 -> pytubefix 라이브러리 사용

- https://pypi.org/project/pytubefix/ 

In [13]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

# [SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략
link = "https://youtu.be/sy2asT2c8FM?si=BJzcXPR6c26Efn2b"

yt = YouTube(link, on_progress_callback = on_progress)
print(yt.title)

ys = yt.streams.get_highest_resolution()

ys = yt.streams.get_audio_only()
filename = ys.download(mp3=True) # pass the parameter mp3=True to save in .mp3

[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략


In [14]:
filename

'/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략.mp3'

## 3 MP3를  WAV 변환
변환을 해야 api 먹힘

In [15]:
from moviepy.editor import AudioFileClip


def convert_mp3_to_wav(filepath):
    # WAV 파일 경로
    wav_file_path = filepath.replace(".mp3", ".wav")

    # MP4 파일 로드
    audio_clip = AudioFileClip(filepath)

    # WAV 형식으로 오디오 추출 및 저장
    audio_clip.write_audiofile(wav_file_path, fps=44100, nbytes=2, codec="pcm_s16le")
    return wav_file_path

In [16]:
wav_file = convert_mp3_to_wav(filename)

MoviePy - Writing audio in /Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략.wav


MoviePy - Done.


## 4 MP3를  chunk 단위로 분할
whisper api의 stt를 위해서

In [17]:
from pydub import AudioSegment

# 오디오 파일 불러오기
audio = AudioSegment.from_file(wav_file, format="wav")

total_length = len(audio)
length_per_chunk = 60 * 1000  # 60초

if not os.path.exists(".tmp"):
    os.mkdir(".tmp")

folder_path = os.path.join(".tmp", wav_file[:-4])

if not os.path.exists(folder_path):
    os.mkdir(folder_path)

chunks = []
for i in range(0, total_length, length_per_chunk):
    chunk_file_path = os.path.join(folder_path, f"{i}.wav")
    audio[i : i + length_per_chunk].export(chunk_file_path, format="wav")
    chunks.append(chunk_file_path)

In [18]:
chunks

['/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/0.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/60000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/120000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/180000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/240000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/300000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/360000.wav',
 '/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 

In [19]:
from datetime import datetime, timedelta
import re


def adjust_timestamps(transcript, minutes=1):
    # Define the regular expression pattern for timestamps
    timestamp_pattern = re.compile(r"(\d{2}:\d{2}:\d{2},\d{3})")

    # Function to add minutes to a timestamp
    def add_minutes(timestamp_str, minutes):
        timestamp = datetime.strptime(timestamp_str, "%H:%M:%S,%f")
        adjusted_timestamp = timestamp + timedelta(minutes=minutes)
        return adjusted_timestamp.strftime("%H:%M:%S,%f")[:-3]

    # Replace timestamps in the transcript
    adjusted_transcript = timestamp_pattern.sub(
        lambda match: add_minutes(match.group(1), minutes), transcript
    )
    return adjusted_transcript

## 5. 텍스트로 추출한거 리스트에 저장

In [20]:
transcripts = []
for i, chunk in enumerate(chunks):
    print(chunk)
    audio_file = open(chunk, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="srt",
        temperature=0.01,
    )
    transcripts.append(adjust_timestamps(transcript, minutes=i))

/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/0.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/60000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/120000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/180000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/240000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/300000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/360000.wav
/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/[SK TECH SUMMIT 2023] RAG를 위한 Retriever 전략/420000.wav
/Users

In [23]:
print(transcripts)

['1\n00:00:00,000 --> 00:00:16,440\n안녕하세요. 저는 오늘 RAG를 위한 리트리버 전략이라는 내용으로 발표를 하도록 하겠습니다.\n\n2\n00:00:16,440 --> 00:00:21,400\n저는 Organize에서 최고 AI 책임자를 맡고 있고요.\n\n3\n00:00:21,400 --> 00:00:27,600\n10몇 년 동안 백엔드 엔지니어로 일하다가 지금은 AI 이렇게 하고 있네요.\n\n4\n00:00:27,600 --> 00:00:34,040\n오늘 말씀드릴 내용은 RAG가 대충 뭔지 간략하게 설명드리고\n\n5\n00:00:34,040 --> 00:00:40,400\n지금 채취 PT나 이런 것 때문에 굉장히 LRM이 발전되어 있어서 말 되게 잘하는데\n\n6\n00:00:40,400 --> 00:00:46,959\n왜 아직도 답변은 제대로 안 나오는가? 그거에 대해서 좀 얘기를 해보려고 합니다.\n\n7\n00:00:46,959 --> 00:00:52,119\n그래서 RAG가 뭔지부터 좀 말씀드리도록 하겠습니다.\n\n8\n00:00:52,160 --> 00:00:55,680\n나온 것은 2021년에 맨 처음 나왔고요.\n\n9\n00:00:55,680 --> 00:00:59,959\n이때는 아직 GPT3 시절이죠.\n\n\n', '1\n00:01:00,000 --> 00:01:09,080\n그래서 이 제네레이션 쪽에서 그렇게 말이 되기는 하고는 있지만 그렇게 놀라운 성능을 발휘하지 못하는 시절이었는데\n\n2\n00:01:09,080 --> 00:01:18,280\n이제 오픈AI에서 모델을 발표하고 난 다음부터는 이 제네레이터의 성능이 엄청나게 높아지니까\n\n3\n00:01:18,280 --> 00:01:26,080\n얘가 이제 이거를 이용해서 어떻게 이제 기업 내부 문서 혹은 나만이 알고 있는 문서들에서 답변을 할 수 있을까\n\n4\n00:01:26,080 --> 00:01:32,759\n하는 고민들을 하기 

## 6. 자막을 위해 영상 시간 분리 깔끔하게 작업

In [21]:
def merge_transcripts(*transcripts):
    merged_transcript = ""
    current_number = 1

    for transcript in transcripts:
        # Split the transcript into segments
        segments = transcript.strip().split("\n\n")
        for segment in segments:
            # Split each segment into lines
            lines = segment.split("\n")
            # Replace the number at the beginning of each segment with the correct sequence number
            lines[0] = str(current_number)
            # Increment the sequence number
            current_number += 1
            # Reassemble the segment
            merged_transcript += "\n".join(lines) + "\n\n"

    return merged_transcript.strip()

## 자막 완성

In [22]:
print(merge_transcripts(*transcripts))

1
00:00:00,000 --> 00:00:16,440
안녕하세요. 저는 오늘 RAG를 위한 리트리버 전략이라는 내용으로 발표를 하도록 하겠습니다.

2
00:00:16,440 --> 00:00:21,400
저는 Organize에서 최고 AI 책임자를 맡고 있고요.

3
00:00:21,400 --> 00:00:27,600
10몇 년 동안 백엔드 엔지니어로 일하다가 지금은 AI 이렇게 하고 있네요.

4
00:00:27,600 --> 00:00:34,040
오늘 말씀드릴 내용은 RAG가 대충 뭔지 간략하게 설명드리고

5
00:00:34,040 --> 00:00:40,400
지금 채취 PT나 이런 것 때문에 굉장히 LRM이 발전되어 있어서 말 되게 잘하는데

6
00:00:40,400 --> 00:00:46,959
왜 아직도 답변은 제대로 안 나오는가? 그거에 대해서 좀 얘기를 해보려고 합니다.

7
00:00:46,959 --> 00:00:52,119
그래서 RAG가 뭔지부터 좀 말씀드리도록 하겠습니다.

8
00:00:52,160 --> 00:00:55,680
나온 것은 2021년에 맨 처음 나왔고요.

9
00:00:55,680 --> 00:00:59,959
이때는 아직 GPT3 시절이죠.

10
00:01:00,000 --> 00:01:09,080
그래서 이 제네레이션 쪽에서 그렇게 말이 되기는 하고는 있지만 그렇게 놀라운 성능을 발휘하지 못하는 시절이었는데

11
00:01:09,080 --> 00:01:18,280
이제 오픈AI에서 모델을 발표하고 난 다음부터는 이 제네레이터의 성능이 엄청나게 높아지니까

12
00:01:18,280 --> 00:01:26,080
얘가 이제 이거를 이용해서 어떻게 이제 기업 내부 문서 혹은 나만이 알고 있는 문서들에서 답변을 할 수 있을까

13
00:01:26,080 --> 00:01:32,759
하는 고민들을 하기 시작했습니다 그래서 이제 사람들이 많이 채택하고 있는 방법이 RAZ 인데요

14
00:01:32

In [17]:
merged_transcript = merge_transcripts(*transcripts)
print(merged_transcript)

1
00:00:00,000 --> 00:00:08,700
2024년 1월 24일 오늘은 생성 AI가 갖는 의미가 단순하게 하나의 트렌드가 아닙니다.

2
00:00:08,700 --> 00:00:15,700
그래서 생성 AI를 기업에서 어떻게 이해하고 준비해야 되고

3
00:00:15,700 --> 00:00:25,000
특히 경영자들이 해야 되는지에 대한 폭넓은 이론서부터 글로벌 현안까지

4
00:00:25,000 --> 00:00:29,299
국내 사타가 인정하는 1인자가 아닌가 싶습니다.

5
00:00:29,299 --> 00:00:32,500
그래서 아주 귀한 장동인 교수님을 모셨고

6
00:00:32,500 --> 00:00:40,299
첫 GPT 기업이 살아남는 법에 대한 내용 외에도 아마 소개를 좀 해주실 것 같은데

7
00:00:40,299 --> 00:00:45,500
첫 GPT를 가지고 어떻게 기업이 살아남아야 되는지

8
00:00:45,500 --> 00:00:50,500
큰 박수로 우리 장동인 교수님의 특강을 시작하도록 하겠습니다.

9
00:00:50,500 --> 00:00:57,000
저는 경영자하고 개발자하고 딱 중간에 있는 사람이에요.

10
00:00:57,000 --> 00:01:00,000
어떻게 보면 교영도 못하고 개발도 못하는

11
00:01:00,000 --> 00:01:03,700
그런 사람일 수도 있고 양쪽을 다 아는 사람일 수도 있고 그런데

12
00:01:03,700 --> 00:01:11,700
어쨌든 이 중간에서 이야기를 하는 게 시간이 지날수록 되게 중요하다는 생각이 점점 듭니다.

13
00:01:11,700 --> 00:01:19,100
왜냐하면 경영하는 사람은 경영만 하고 개발하는 사람은 개발만 하는 그런 시대는 이제 지났잖아요.

14
00:01:19,100 --> 00:01:20,700
양쪽이 양쪽을 다 알아야죠.

15
00:01:20,700 --> 00:01:25,580
그렇기 때

In [18]:
with open("sample.srt", "w") as f:
    f.write(merged_transcript)

In [20]:
import tiktoken
from openai import OpenAI
import re

client = OpenAI()

MAX_COMPLETION_TOKENS = 4096
MAX_CONTEXT_LENGTH = 8192  # GPT-4의 일반적인 컨텍스트 길이


def count_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))


def chunk_text(text: str, max_chunk_tokens: int = 3000) -> list[str]:
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in text.split(". "):
        sentence_tokens = count_tokens(sentence)
        if current_tokens + sentence_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_tokens = sentence_tokens
        else:
            current_chunk += sentence + ". "
            current_tokens += sentence_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


def process_chunk(chunk: str, system_prompt: str) -> str:
    prompt_tokens = count_tokens(system_prompt) + count_tokens(chunk)
    max_response_tokens = min(
        MAX_COMPLETION_TOKENS, MAX_CONTEXT_LENGTH - prompt_tokens - 100
    )

    if max_response_tokens <= 0:
        return "Error: 입력 텍스트가 너무 깁니다."

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            temperature=0.1,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chunk},
            ],
            max_tokens=max_response_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: API 호출 중 오류 발생 - {str(e)}"


def format_summary(summary: str) -> str:
    lines = summary.split("\n")
    formatted_lines = []
    for line in lines:
        if line.strip():
            match = re.match(r"^(\d{2}:\d{2})\s+(.+)$", line)
            if match:
                time, content = match.groups()
                if not any(
                    emoji in content
                    for emoji in [
                        "🔶",
                        "👋",
                        "👀",
                        "💡",
                        "🎯",
                        "📊",
                        "🔑",
                        "💻",
                        "🤔",
                        "📝",
                    ]
                ):
                    content = "🔶 " + content
                formatted_lines.append(f"{time} {content}")
            else:
                formatted_lines.append(line)
    return "\n".join(formatted_lines)


def post_processing(instruction: str) -> str:
    system_prompt = """
    당신은 비디오 자막을 분석하고 요약하는 AI 어시스턴트입니다. 주어진 자막 정보를 바탕으로 다음 작업을 수행해야 합니다:
    1. 중요한 주제를 시간 순서에 맞게 선정하세요.
    2. 각 주제에 대해 한 줄 요약을 작성하세요.
    3. 각 주제가 시작되는 시간을 mm:ss(분:초) 형식으로 기록하세요.
    4. 각 요약 문장에 적합한 이모지를 추가하세요.
    5. 요약은 한글로 작성하세요.
    6. 주제는 최대한 많이 추출해 주세요.
    7. 시간 표기에 오류가 없도록 주의하세요.

    출력 형식:
    mm:ss 🔶 주제에 대한 한 줄 요약
    """

    chunks = chunk_text(instruction)
    summaries = []

    print(f"총 {len(chunks)}개의 청크로 나누어 처리합니다.\n")

    for i, chunk in enumerate(chunks):
        print(f"\n청크 {i+1}/{len(chunks)} 처리 중...\n")
        summary = process_chunk(chunk, system_prompt)
        summaries.append(summary)
        print(summary)
        print("\n")

    final_summary = "\n".join(summaries)
    formatted_summary = format_summary(final_summary)

    return formatted_summary

In [21]:
instruction = f"""주어진 비디오 "자막정보" 를 바탕으로 [요청사항]을 차례대로 수행해주세요.
[자막정보]
{merged_transcript}

[요청사항]
1. 주어진 [자막정보]에서 중요한 주제를 시간 순서에 맞게 선정하고, 주제가 시작되는 시간을 기록해주세요.
2. 주제는 한 줄 요약을 작성하고, 자막에서 주제가 시작되는 시간을 mm:ss(분:초) 형식으로 작성하세요. (예: 00:05 👋 자막 생성 기능에 대한 소개)
3. 요약은 한글로 작성해주세요.
4. 각 문장에 적합한 emoji를 최대한 활용해 주세요

[출력예시]
00:12 👋 GPTs 의 주요 개념에 대하여 소개해요
03:13 👀 GPTs 의 장단점과 활용 사례에 대하여 자세히 알아봐요

[주의사항]
- 주제는 최대한 많이 추출해 주세요.
- 시간표기에 오류가 없도록 주의해 주세요. 분:초 형식으로 작성합니다. (예: 00:12)

run step-by-step. Take a deep breath. You can do it!
"""
summary_output = post_processing(instruction)

총 16개의 청크로 나누어 처리합니다.


청크 1/16 처리 중...

00:00 🔶 생성 AI의 중요성: 생성 AI는 단순한 트렌드가 아닌 중요한 기술입니다. 🤖

00:08 🔶 기업의 생성 AI 준비: 기업이 생성 AI를 이해하고 준비하는 방법에 대해 논의합니다. 🏢

00:15 🔶 경영자와 생성 AI: 경영자들이 생성 AI를 어떻게 활용해야 하는지에 대한 이론과 현안을 다룹니다. 📚

00:25 🔶 장동인 교수 소개: 장동인 교수님이 초청되어 특강을 진행합니다. 🎓

00:32 🔶 첫 GPT 기업 생존법: 첫 GPT를 활용해 기업이 생존하는 방법을 소개합니다. 💼

00:50 🔶 경영자와 개발자의 중간 역할: 경영자와 개발자 사이의 중간 역할이 중요하다고 강조합니다. 🔄

01:11 🔶 경영자와 개발자의 협력 필요성: 경영자와 개발자가 서로의 역할을 이해해야 한다고 설명합니다. 🤝

01:34 🔶 AI 이해의 어려움: AI를 이해하는 것이 어렵다는 점을 지적합니다. 🧠

02:00 🔶 AI 시대의 역할: AI 시대에 경영자와 개발자의 역할을 메꿔주는 것이 중요하다고 말합니다. 🌐

02:06 🔶 책 소개: '기업이 살아남는 법'이라는 책을 통해 경영자와 개발자의 결합을 시도합니다. 📖

02:18 🔶 GPT의 발전: GPT 3.5, GPT 4, 그리고 곧 나올 GPT 5에 대해 언급합니다. 🚀

02:30 🔶 GPT의 활용과 변화: GPT가 기업에 미치는 영향과 활용 방법에 대해 논의합니다. 🔄

02:39 🔶 GPT 도입의 고민: 많은 기업들이 GPT 도입을 고민하고 있는 상황을 설명합니다. 🤔

02:50 🔶 GPT의 한계: GPT의 한계를 명확히 이해해야 한다고 강조합니다. 🚧

03:00 🔶 콜센터에서의 GPT 활용: 콜센터에서 GPT를 활용하는 방법과 그 한계를 설명합니다. 📞

03:26 🔶 법규 준수의 어려움: GPT가 법규를 준수하는 데 어려움이 있다는 점을 지적합니다. ⚖️

03:46 🔶 콜센터의 책임: 콜센터에서 GPT가 고객에게 하

In [22]:
print(summary_output)

00:00 🔶 생성 AI의 중요성: 생성 AI는 단순한 트렌드가 아닌 중요한 기술입니다. 🤖
00:08 🔶 기업의 생성 AI 준비: 기업이 생성 AI를 이해하고 준비하는 방법에 대해 논의합니다. 🏢
00:15 🔶 경영자와 생성 AI: 경영자들이 생성 AI를 어떻게 활용해야 하는지에 대한 이론과 현안을 다룹니다. 📚
00:25 🔶 장동인 교수 소개: 장동인 교수님이 초청되어 특강을 진행합니다. 🎓
00:32 🔶 첫 GPT 기업 생존법: 첫 GPT를 활용해 기업이 생존하는 방법을 소개합니다. 💼
00:50 🔶 경영자와 개발자의 중간 역할: 경영자와 개발자 사이의 중간 역할이 중요하다고 강조합니다. 🔄
01:11 🔶 경영자와 개발자의 협력 필요성: 경영자와 개발자가 서로의 역할을 이해해야 한다고 설명합니다. 🤝
01:34 🔶 AI 이해의 어려움: AI를 이해하는 것이 어렵다는 점을 지적합니다. 🧠
02:00 🔶 AI 시대의 역할: AI 시대에 경영자와 개발자의 역할을 메꿔주는 것이 중요하다고 말합니다. 🌐
02:06 🔶 책 소개: '기업이 살아남는 법'이라는 책을 통해 경영자와 개발자의 결합을 시도합니다. 📖
02:18 🔶 GPT의 발전: GPT 3.5, GPT 4, 그리고 곧 나올 GPT 5에 대해 언급합니다. 🚀
02:30 🔶 GPT의 활용과 변화: GPT가 기업에 미치는 영향과 활용 방법에 대해 논의합니다. 🔄
02:39 🔶 GPT 도입의 고민: 많은 기업들이 GPT 도입을 고민하고 있는 상황을 설명합니다. 🤔
02:50 🔶 GPT의 한계: GPT의 한계를 명확히 이해해야 한다고 강조합니다. 🚧
03:00 🔶 콜센터에서의 GPT 활용: 콜센터에서 GPT를 활용하는 방법과 그 한계를 설명합니다. 📞
03:26 🔶 법규 준수의 어려움: GPT가 법규를 준수하는 데 어려움이 있다는 점을 지적합니다. ⚖️
03:46 🔶 콜센터의 책임: 콜센터에서 GPT가 고객에게 하는 말을 기업이 책임져야 한다는 점을 강조합니다. 📋
04:00 🔶 Hallucination 문제: G